In [166]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [167]:
titanicData = pd.read_csv('http://bit.ly/kaggletrain', index_col = None)
titanicData

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [168]:
titanicData.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [169]:
titanicData.shape

(891, 12)

In [170]:
titanicData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [171]:
cols = ["Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Embarked"]
titanicData_X = titanicData[cols]
titanicData_y = titanicData["Survived"]
titanicData_X

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,S
...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,S
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,S
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,C


In [172]:
titanicData_y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [173]:
titanicData_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 48.9+ KB


### Required transformations
- Pclass: None
- Name: CountVectorizer
- Sex: Categorical to Numerical
- Age: Missing value, To numerical
- Embarked: To numerical

In [174]:
knn = KNeighborsClassifier(n_neighbors = 8)
X_train, X_test, y_train, y_test = train_test_split(
    titanicData_X, titanicData_y,
    random_state = 40, test_size = 0.2
)

In [175]:
X_train.shape

(712, 7)

In [176]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 661 to 326
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Name      712 non-null    object 
 2   Sex       712 non-null    object 
 3   Age       573 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Embarked  711 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 44.5+ KB


In [177]:
y_train.shape

(712,)

In [178]:
countVectorizer = CountVectorizer()
oneHotEncoder = OneHotEncoder()

In [179]:
columnTransformer = make_column_transformer(
    (countVectorizer, "Name"),
    (oneHotEncoder, ["Sex"]),
    remainder = "passthrough",
    n_jobs = -1
)

In [180]:
pipeLine = make_pipeline(columnTransformer, knn)

In [181]:
k_range = list(range(1, 31))
params = {"kneighborsclassifier__n_neighbors": k_range}

In [182]:
gridSearchCV = GridSearchCV(
    pipeLine, params, cv = 5, scoring = "accuracy", n_jobs = -1
)

In [183]:
gridSearchCV.fit(X_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.